In [ ]:
# --- imports & config ---
import json, numpy as np, pandas as pd
from pathlib import Path

DATA_PATH = "dataset_Recomendacion_villa_de_leyva_eleccion_v2_compania.csv"   # ajústalo
CAT_PATH  = "catalogo_vdl_lugares_unico"               # ajústalo
SEP, ENC  = ";", "utf-8-sig"
RANDOM_SEED = 42
K_LIST = [3, 5, 10]
TEST_USER_FRAC = 0.20     # % de usuarios para hold-out honesto

# --- normalizador de encabezados con caracteres raros ---
def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    ren = {
        "compa¤¡a_viaje": "compañia_viaje",
        "‚poca_visita": "epoca_visita",
    }
    ren = {k:v for k,v in ren.items() if k in df.columns}
    return df.rename(columns=ren)

# --- métricas Top-K ---
def _dcg_at_k(rels): return float(np.sum([r/np.log2(i+2) for i,r in enumerate(rels)]))

def recall_at_k(g, k, score_col, rel_col):
    g = g.sort_values(score_col, ascending=False)
    topk = g.head(k)
    tot = g[rel_col].sum()
    return float("nan") if tot==0 else float(topk[rel_col].sum()/tot)

def ndcg_at_k(g, k, score_col, rel_col):
    g = g.sort_values(score_col, ascending=False)
    dcg  = _dcg_at_k(g.head(k)[rel_col].tolist())
    idcg = _dcg_at_k(sorted(g[rel_col].tolist(), reverse=True)[:k])
    return float("nan") if idcg==0 else float(dcg/idcg)

def coverage_at_k(df, k, score_col, item_col="nombre_sitio"):
    topk = (df.sort_values(["id_usuario", score_col], ascending=[True, False])
              .groupby("id_usuario").head(k))
    return float(topk[item_col].nunique() / df[item_col].nunique())


In [4]:
# --- carga ---
df = pd.read_csv(DATA_PATH, sep=SEP, encoding=ENC)
df = normalize_columns(df)
# === ARMONIZACIÓN DE COLUMNAS (poner justo después de normalize_columns(df)) ===
import pandas as pd, unicodedata, re

def _norm(s: str) -> str:
    s = str(s or "")
    s = unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode("ascii")
    s = s.lower().strip()
    s = re.sub(r"[\s\-]+", "_", s)
    s = re.sub(r"__+", "_", s)
    return s

# mapa normalizado -> nombre real
norm2real = {_norm(c): c for c in df.columns}

def rename_if_present(target: str, candidates):
    """Renombra la primera coincidencia encontrada (robusto a acentos, espacios y guiones)."""
    if target in df.columns:
        return
    for cand in candidates:
        # 1) match exacto
        if cand in df.columns:
            df.rename(columns={cand: target}, inplace=True); return
        # 2) match por normalización
        nc = _norm(cand)
        if nc in norm2real:
            df.rename(columns={norm2real[nc]: target}, inplace=True); return

# 0) compañía (por si el normalizador quitó el acento)
rename_if_present("compania_viaje", ["compañia_viaje","compania_viaje"])

# 1) sinónimos fundamentales
rename_if_present("costo_entrada",      ["costo_entrada","costo","precio_entrada","valor_entrada","entrada","costoentrada","precio"])
rename_if_present("afluencia_promedio", ["afluencia_promedio","afluencia","nivel_afluencia","popularidad"])
rename_if_present("duracion_esperada",  ["duracion_esperada","duracion","tiempo_recorrido","tiempo_estadia","tiempo_promedio"])
rename_if_present("admite_mascotas",    ["admite_mascotas","mascotas","pet_friendly","permite_mascotas"])

# 2) tipado seguro
num_fix = {
    "costo_entrada": 0,
    "presupuesto_estimado": None,
    "afluencia_promedio": 3,
    "duracion_esperada": 60,
    "admite_mascotas": 0,
}
for col, fill in num_fix.items():
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        if fill is not None:
            df[col] = df[col].fillna(fill)
        if col == "afluencia_promedio":
            df[col] = df[col].clip(1,5).round().astype(int)
        if col == "admite_mascotas":
            df[col] = df[col].fillna(0).astype(int)

# 3) listas finales (sin acento en 'compania_viaje')
CAT_COLS = [
    "nacionalidad","origen","tipo_turista_preferido","compania_viaje",
    "restricciones_movilidad","nombre_sitio","tipo_sitio","accesibilidad_general",
    "idioma_info","ubicacion_geografica","clima_predominante","epoca_visita"
]
NUM_COLS = [
    "edad","frecuencia_viaje","presupuesto_estimado","sitios_visitados",
    "calificacion_sitios_previos","tiempo_estancia_promedio","costo_entrada",
    "afluencia_promedio","duracion_esperada","admite_mascotas"
]

missing = [c for c in (CAT_COLS+NUM_COLS) if c not in df.columns]
if missing:
    print("⚠️ Faltan columnas tras armonización:", missing)

# Depuración útil (puedes comentar estas dos líneas si quieres)
print("Columnas disponibles:", sorted(df.columns.tolist())[:20], "...")
assert "costo_entrada" in df.columns, "No se logró crear/renombrar 'costo_entrada'. Revisa el listado impreso."
# === FIN ARMONIZACIÓN ===

# quita posibles fugas
if "sitio_recomendado" in df.columns:
    df = df.drop(columns=["sitio_recomendado"])

# columnas por tipo (ajústalas si cambian en tu dataset)
CAT_COLS = [
    "nacionalidad","origen","tipo_turista_preferido","compañia_viaje",
    "restricciones_movilidad","nombre_sitio","tipo_sitio","accesibilidad_general",
    "idioma_info","ubicacion_geografica","clima_predominante","epoca_visita"
]
NUM_COLS = [
    "edad","frecuencia_viaje","presupuesto_estimado","sitios_visitados",
    "calificacion_sitios_previos","tiempo_estancia_promedio","costo_entrada",
    "afluencia_promedio","duracion_esperada","admite_mascotas"
]

# --- features de interacción usuario×sitio ---
AFINIDAD = {
    "cultural": {"museo":0.9,"histórico":0.9,"religioso":0.7,"arquitectura":0.85,"museo_religioso":0.8,"arqueologico":0.85,"plaza":0.7},
    "naturaleza": {"natural":0.95,"senderismo":0.9,"mirador":0.8,"parque_urbano":0.6},
    "aventura": {"aventura":0.95,"senderismo":0.85,"parque_tematico":0.7},
    "gastronómico": {"gastronomico":0.95,"enoturismo":0.9,"artesanal":0.6,"plaza":0.6},
    "relax_fotografía": {"mirador":0.9,"plaza":0.8,"arquitectura":0.8,"natural":0.75},
}

def make_features(X: pd.DataFrame) -> pd.DataFrame:
    X = X.copy()
    # costo relativo al 15% del presupuesto diario
    X["ratio_costo_presu"] = (X["costo_entrada"] / (X["presupuesto_estimado"]*0.15)).clip(0, 3)
    # afinidad perfil-tipo_sitio
    X["afinidad_tipo"] = X.apply(lambda r: AFINIDAD.get(r["tipo_turista_preferido"],{}).get(r["tipo_sitio"],0.5), axis=1)
    # cruces categóricos (ayuda a modelos lineales y deja explícita la interacción)
    X["x_tipoTur__tipoSit"] = X["tipo_turista_preferido"] + "×" + X["tipo_sitio"]
    X["x_epoca__tipoSit"]    = X["epoca_visita"] + "×" + X["tipo_sitio"]
    return X

df = make_features(df)

# añade variable binaria "like" para clasificación
df["y_like"] = (df["rating_usuario"] >= 4.0).astype(int)

# columnas extendidas
CAT_COLS_X = CAT_COLS + ["x_tipoTur__tipoSit","x_epoca__tipoSit"]
NUM_COLS_X = NUM_COLS + ["ratio_costo_presu","afinidad_tipo"]


⚠️ Faltan columnas tras armonización: ['nacionalidad', 'origen', 'tipo_turista_preferido', 'compania_viaje', 'restricciones_movilidad', 'nombre_sitio', 'tipo_sitio', 'accesibilidad_general', 'idioma_info', 'ubicacion_geografica', 'clima_predominante', 'epoca_visita', 'edad', 'frecuencia_viaje', 'presupuesto_estimado', 'sitios_visitados', 'calificacion_sitios_previos', 'tiempo_estancia_promedio', 'costo_entrada', 'afluencia_promedio', 'duracion_esperada', 'admite_mascotas']
Columnas disponibles: ['id_usuario,edad,nacionalidad,origen,tipo_turista_preferido,compañia_viaje,frecuencia_viaje,restricciones_movilidad,presupuesto_estimado,sitios_visitados,calificacion_sitios_previos,tiempo_estancia_promedio,nombre_sitio,tipo_sitio,costo_entrada,afluencia_promedio,duracion_esperada,accesibilidad_general,admite_mascotas,idioma_info,ubicacion_geografica,clima_predominante,epoca_visita,sitio_recomendado,rating_usuario'] ...


AssertionError: No se logró crear/renombrar 'costo_entrada'. Revisa el listado impreso.

In [ ]:
# === FEATURES ROBUSTAS (reemplaza tu implementación) ===
import unicodedata, pandas as pd

def _norm_txt(s):
    s = str(s or "").lower()
    s = unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode("ascii")
    return s

AFINIDAD = {
    "cultural":   {"museo":0.9,"historico":0.9,"religioso":0.7,"arquitectura":0.85,"museo_religioso":0.8,"arqueologico":0.85,"plaza":0.7},
    "naturaleza": {"natural":0.95,"senderismo":0.9,"mirador":0.8,"parque_urbano":0.6},
    "aventura":   {"aventura":0.95,"senderismo":0.85,"parque_tematico":0.7},
    "gastronomia":{"gastronomico":0.95,"enoturismo":0.9,"artesanal":0.6,"plaza":0.6},
    "relax":      {"mirador":0.9,"plaza":0.8,"arquitectura":0.8,"natural":0.75},
    "fotografia": {"mirador":0.9,"plaza":0.8,"arquitectura":0.8,"natural":0.75},
}
_alias_tipo_tur = {"gastronomico":"gastronomia","gastronomia":"gastronomia","relax_fotografia":"relax"}

def make_features(X: pd.DataFrame) -> pd.DataFrame:
    X = X.copy()
    # ratio costo/presupuesto (a prueba de faltantes)
    presu = X.get("presupuesto_estimado", 1).astype(float).replace(0, 1.0)
    costo = X.get("costo_entrada", 0).astype(float).fillna(0.0)
    X["ratio_costo_presu"] = (costo / (presu * 0.15)).clip(0, 3)

    # afinidad perfil × tipo_sitio (normaliza texto y acentos)
    def _afin(r):
        tt = _norm_txt(r.get("tipo_turista_preferido",""))
        ts = _norm_txt(r.get("tipo_sitio",""))
        tt = _alias_tipo_tur.get(tt, tt)
        return AFINIDAD.get(tt, {}).get(ts, 0.5)
    X["afinidad_tipo"] = X.apply(_afin, axis=1)

    # cruces categóricos
    X["x_tipoTur__tipoSit"] = X["tipo_turista_preferido"].astype(str) + "×" + X["tipo_sitio"].astype(str)
    X["x_epoca__tipoSit"]   = X["epoca_visita"].astype(str) + "×" + X["tipo_sitio"].astype(str)
    return X
# === FIN FEATURES ROBUSTAS ===


In [ ]:
# === BLOQUE DE ARMONIZACIÓN (insertado) ===
# Coloca este bloque inmediatamente DESPUÉS de normalize_columns(df) y ANTES de make_features(df)

import pandas as pd
import unicodedata

# 1) Si tu normalizador quita acentos, renombra 'compañia_viaje' -> 'compania_viaje'
if "compañia_viaje" in df.columns and "compania_viaje" not in df.columns:
    df = df.rename(columns={"compañia_viaje": "compania_viaje"})

# 2) Sinónimos -> nombre esperado
SYNONYMS = {
    "costo_entrada":       ["costo_entrada","costo","precio_entrada","valor_entrada","entrada"],
    "afluencia_promedio":  ["afluencia_promedio","afluencia","nivel_afluencia","popularidad"],
    "duracion_esperada":   ["duracion_esperada","duracion","tiempo_recorrido","tiempo_estadia","tiempo_promedio"],
    "admite_mascotas":     ["admite_mascotas","pet_friendly","mascotas","permite_mascotas"],
}
for target, opts in SYNONYMS.items():
    if target not in df.columns:
        for opt in opts:
            if opt in df.columns:
                df = df.rename(columns={opt: target})
                break

# 3) Tipos numéricos y rangos plausibles
if "costo_entrada" in df.columns:
    df["costo_entrada"] = pd.to_numeric(df["costo_entrada"], errors="coerce").fillna(0)
if "presupuesto_estimado" in df.columns:
    df["presupuesto_estimado"] = pd.to_numeric(df["presupuesto_estimado"], errors="coerce")\
                                    .fillna(df["presupuesto_estimado"].median())
if "afluencia_promedio" in df.columns:
    df["afluencia_promedio"] = pd.to_numeric(df["afluencia_promedio"], errors="coerce")\
                                  .fillna(3).clip(1,5).astype(int)
if "duracion_esperada" in df.columns:
    df["duracion_esperada"] = pd.to_numeric(df["duracion_esperada"], errors="coerce").fillna(60)
if "admite_mascotas" in df.columns:
    df["admite_mascotas"] = pd.to_numeric(df["admite_mascotas"], errors="coerce").fillna(0).astype(int)

# 4) Actualiza listas de features (versión sin acento en 'compania_viaje')
CAT_COLS = [
    "nacionalidad","origen","tipo_turista_preferido","compania_viaje",
    "restricciones_movilidad","nombre_sitio","tipo_sitio","accesibilidad_general",
    "idioma_info","ubicacion_geografica","clima_predominante","epoca_visita"
]
NUM_COLS = [
    "edad","frecuencia_viaje","presupuesto_estimado","sitios_visitados",
    "calificacion_sitios_previos","tiempo_estancia_promedio","costo_entrada",
    "afluencia_promedio","duracion_esperada","admite_mascotas"
]

missing = [c for c in (CAT_COLS+NUM_COLS) if c not in df.columns]
if missing:
    print("⚠️ Faltan columnas tras armonización:", missing)
# === FIN BLOQUE DE ARMONIZACIÓN ===


In [4]:
rng = np.random.default_rng(RANDOM_SEED)
users = df["id_usuario"].drop_duplicates().to_numpy()
test_users = set(rng.choice(users, size=int(len(users)*TEST_USER_FRAC), replace=False))

train_df = df[~df["id_usuario"].isin(test_users)].reset_index(drop=True)
test_df  = df[ df["id_usuario"].isin(test_users)].reset_index(drop=True)

print("Usuarios train/test:", train_df["id_usuario"].nunique(), test_df["id_usuario"].nunique())
print("Filas train/test:", train_df.shape, test_df.shape)


Usuarios train/test: 9586 2396
Filas train/test: (63955, 29) (16045, 29)


In [5]:
# ==== Multi-hot para columnas con múltiples valores ====
CATS_USER  = ["cultural","naturaleza","aventura","gastronomico","relax_fotografía"]
CATS_SITIO = ["plaza","centro_historico","histórico","museo","arquitectura","naturaleza",
              "senderismo","mirador","aventura","gastronomico","enoturismo","artesanal",
              "religioso","arqueologico","parque_tematico"]

def _split_tags(x):
    if x is None: 
        return set()
    if isinstance(x, float):
        try:
            import math
            if math.isnan(x): 
                return set()
        except Exception:
            pass
    return {t.strip().lower().replace(" ", "_") for t in str(x).split(";") if t.strip()}

def add_multihot_cols(df):
    # 1) preferencias del usuario (tipo_turista_preferido)
    tags_u = df["tipo_turista_preferido"].fillna("").apply(_split_tags)
    for t in CATS_USER:
        df[f"pref_{t}"] = tags_u.apply(lambda s: int(t in s)).astype("int8")

    # 2) tipos/categorías del sitio (tipo_sitio multi)
    tags_s = df["tipo_sitio"].fillna("").apply(_split_tags)
    for t in CATS_SITIO:
        df[f"sitio_{t}"] = tags_s.apply(lambda s: int(t in s)).astype("int8")

    return df


In [6]:
from pycaret.classification import (
    setup, compare_models, tune_model, blend_models,
    finalize_model, predict_model, pull, save_model
)
import pandas as pd
import numpy as np

# 1) Copias profundas y tipos explícitos (evita vistas read-only)
train_df = train_df.copy(deep=True)

# Asegura que id_usuario esté como string (pero NO como feature)
train_df["id_usuario"] = train_df["id_usuario"].astype("string")

for c in CAT_COLS_X:
    if c in train_df.columns:
        train_df[c] = train_df[c].astype("string").copy()

for c in NUM_COLS_X:
    if c in train_df.columns:
        train_df[c] = pd.to_numeric(train_df[c], errors="coerce").astype("float64").copy()

# 2) Setup — sin SMOTE y sin paralelismo en la primera corrida
setup_cls = setup(
    data = train_df[["id_usuario"] + CAT_COLS_X + NUM_COLS_X + ["y_like"]].copy(),
    target = "y_like",
    fold = 5,
    fold_strategy = "groupkfold",
    fold_groups = "id_usuario",          # usa el id para agrupar
    categorical_features = CAT_COLS_X,
    ignore_features = ["id_usuario"],     # << clave: no lo pases como feature
    remove_multicollinearity = True,
    multicollinearity_threshold = 0.95,
    imputation_type = "simple",
    fix_imbalance = False,                # << desactivar por ahora
    n_jobs = 1,                           # << sin paralelismo (evita bug loky/writeable)
    verbose = False
)

best3 = compare_models(n_select=3, sort="AUC")
tuned = [tune_model(m, optimize="AUC") for m in best3]
blend = blend_models(tuned)
final_cls = finalize_model(blend)
save_model(final_cls, "modelo_cls_like_v2")

# Si esto corre OK, ya puedes volver a activar fix_imbalance=True y/o subir n_jobs




,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9044,0.9317,0.7310,0.7880,0.7584,0.6989,0.6997,10.0940
ridge,Ridge Classifier,0.9033,0.9301,0.7034,0.8013,0.7491,0.6895,0.6918,2.9020
ada,Ada Boost Classifier,0.9023,0.9300,0.7468,0.7701,0.7583,0.6970,0.6972,5.1520
lda,Linear Discriminant Analysis,0.9031,0.9299,0.7339,0.7812,0.7567,0.6963,0.6969,2.7260
lightgbm,Light Gradient Boosting Machine,0.9035,0.9299,0.7258,0.7876,0.7554,0.6954,0.6963,2.9140
catboost,CatBoost Classifier,0.9037,0.9290,0.7304,0.7856,0.7569,0.6970,0.6977,22.3880
rf,Random Forest Classifier,0.9016,0.9205,0.7186,0.7844,0.7500,0.6889,0.6900,7.7080
et,Extra Trees Classifier,0.8982,0.9157,0.7129,0.7738,0.7420,0.6788,0.6797,8.9840
lr,Logistic Regression,0.8965,0.8971,0.6600,0.8010,0.7236,0.6606,0.6654,8.6900
dt,Decision Tree Classifier,0.8525,0.7795,0.6534,0.6376,0.6453,0.5522,0.5524,3.4260


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9074,0.9375,0.7555,0.7833,0.7691,0.7113,0.7114
1,0.9045,0.9285,0.7274,0.7969,0.7606,0.7011,0.7022
2,0.8993,0.9299,0.7094,0.7930,0.7489,0.6861,0.6878
3,0.9014,0.9233,0.7224,0.7734,0.7471,0.6859,0.6865
4,0.9073,0.9356,0.7275,0.7937,0.7591,0.7019,0.7029
Mean,0.9040,0.9310,0.7284,0.7881,0.7570,0.6973,0.6982
Std,0.0032,0.0051,0.0150,0.0086,0.0081,0.0099,0.0096


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9073,0.9373,0.7287,0.7995,0.7624,0.7050,0.7062
1,0.9035,0.9265,0.6974,0.8132,0.7509,0.6915,0.6946
2,0.8980,0.9290,0.6857,0.8040,0.7401,0.6772,0.6805
3,0.9024,0.9242,0.6942,0.7956,0.7414,0.6816,0.6840
4,0.9045,0.9358,0.6952,0.8028,0.7452,0.6868,0.6895
Mean,0.9031,0.9306,0.7002,0.8030,0.7480,0.6884,0.6910
Std,0.0030,0.0052,0.0148,0.0059,0.0081,0.0096,0.0090


Fitting 5 folds for each of 10 candidates, totalling 50 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9066,0.9354,0.7429,0.7877,0.7646,0.7065,0.7069
1,0.9045,0.9273,0.7209,0.8012,0.7590,0.6996,0.7011
2,0.8977,0.9293,0.7036,0.7903,0.7444,0.6807,0.6825
3,0.9018,0.9214,0.7169,0.7786,0.7465,0.6857,0.6866
4,0.9067,0.9347,0.7158,0.7989,0.7551,0.6977,0.6993
Mean,0.9035,0.9296,0.7200,0.7913,0.7539,0.6940,0.6953
Std,0.0034,0.0051,0.0128,0.0081,0.0076,0.0094,0.0092


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9086,0.0000,0.7505,0.7912,0.7704,0.7134,0.7138
1,0.9050,0.0000,0.7300,0.7971,0.7621,0.7029,0.7039
2,0.9005,0.0000,0.7189,0.7920,0.7537,0.6915,0.6928
3,0.9019,0.0000,0.7247,0.7744,0.7487,0.6879,0.6885
4,0.9052,0.0000,0.7164,0.7916,0.7521,0.6937,0.6950
Mean,0.9042,0.0000,0.7281,0.7893,0.7574,0.6979,0.6988
Std,0.0028,0.0000,0.0122,0.0077,0.0078,0.0092,0.0090


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['edad', 'frecuencia_viaje',
                                              'presupuesto_estimado',
                                              'sitios_visitados',
                                              'calificacion_sitios_previos',
                                              'tiempo_estancia_promedio',
                                              'costo_entrada',
                                              'afluencia_promedio',
                                              'duracion_esperada',
                                              'admite_mascotas',
                                              'ratio_costo_presu',
                                              'afinidad_tipo'],
                                     transf...
                                                RidgeClassifier(alph

In [7]:
# ================= PARCHE REGRESIÓN =================
from pycaret.regression import (
    setup as setup_reg, compare_models as compare_reg, tune_model as tune_reg,
    blend_models as blend_reg, finalize_model as finalize_reg, predict_model as predict_reg,
    pull as pull_reg, save_model as save_reg
)
import pandas as pd, numpy as np

# 0) Preparar train con copias profundas y tipos explícitos
train_reg = train_df[["id_usuario"] + CAT_COLS_X + NUM_COLS_X + ["rating_usuario"]].copy(deep=True)

# Asegurar tipos
train_reg["id_usuario"] = train_reg["id_usuario"].astype("string")
for c in CAT_COLS_X:
    if c in train_reg.columns:
        train_reg[c] = train_reg[c].astype("string").copy()
for c in NUM_COLS_X:
    if c in train_reg.columns:
        train_reg[c] = pd.to_numeric(train_reg[c], errors="coerce").astype("float64").copy()

# Target numérico y sin NaNs / inf
train_reg["rating_usuario"] = pd.to_numeric(train_reg["rating_usuario"], errors="coerce").astype("float64")
train_reg = train_reg.replace([np.inf, -np.inf], np.nan)

# 1) Setup — sin paralelismo, ignorando id_usuario (y opcionalmente nombre_sitio)
IGNORE_FEATS = ["id_usuario"]  # agrega "nombre_sitio" si quieres evitar OHE de alta cardinalidad
setup_reg(
    data = train_reg,
    target = "rating_usuario",
    session_id = RANDOM_SEED,
    fold = 5,
    fold_strategy = "groupkfold",
    fold_groups = "id_usuario",
    categorical_features = CAT_COLS_X,
    ignore_features = IGNORE_FEATS,
    remove_multicollinearity = True,
    multicollinearity_threshold = 0.95,
    imputation_type = "simple",
    n_jobs = 1,            # evita errores de loky/writeable en 1ra pasada
    verbose = False
)

# 2) Benchmark, tuning y ensamble
best3r = compare_reg(n_select=3, sort="RMSE")
lb_reg = pull_reg(); lb_reg.to_csv("leaderboard_regresion.csv", index=False, encoding="utf-8-sig")

tunedr = [tune_reg(m, optimize="RMSE") for m in best3r]
blendr = blend_reg(tunedr)
final_reg = finalize_reg(blendr)
save_reg(final_reg, "modelo_reg_rating_v2")

# 3) Evaluación en TEST (alineamiento seguro para métricas Top-K)
# --- evaluación en TEST (sin columnas duplicadas) ---
Xtest = test_df[["id_usuario"] + CAT_COLS_X + NUM_COLS_X].copy(deep=True)
Xtest["id_usuario"] = Xtest["id_usuario"].astype("string")
for c in CAT_COLS_X:
    if c in Xtest.columns:
        Xtest[c] = Xtest[c].astype("string")
for c in NUM_COLS_X:
    if c in Xtest.columns:
        Xtest[c] = pd.to_numeric(Xtest[c], errors="coerce").astype("float64")

# Guardamos ytest SOLO con la verdad (rating_usuario) para evitar duplicados
ytest = test_df[["rating_usuario"]].reset_index(drop=True)

# Predicción (predict_model devuelve X original + columna 'prediction_label')
pred = predict_reg(final_reg, data=Xtest).reset_index(drop=True)
pred = pred.rename(columns={"prediction_label": "rating_prev"})

# Nos quedamos con UNA SOLA columna de id y de item:
# - Si 'nombre_sitio' está en pred (porque lo usamos como feature), lo conservamos de ahí.
# - Si NO está (porque lo ignoraste), lo traemos desde test_df.
cols_keep = ["id_usuario", "rating_prev"]
if "nombre_sitio" in pred.columns:
    cols_keep.append("nombre_sitio")
test_pred = pred[cols_keep].copy()

if "nombre_sitio" not in test_pred.columns:
    test_pred = pd.concat([test_pred, test_df[["nombre_sitio"]].reset_index(drop=True)], axis=1)

# Añadimos la verdad del rating sin duplicar id/ítem
test_pred["rating_usuario"] = ytest["rating_usuario"].values
test_pred["y_true_rel"] = (test_pred["rating_usuario"] >= 4.0).astype(int)

# (opcional) sanity check: que sólo haya UNA columna 'id_usuario' y 'nombre_sitio'
# print([c for c in test_pred.columns if c == "id_usuario"])
# print([c for c in test_pred.columns if c == "nombre_sitio"])

# --- Métricas Top-K ---
metrics_reg = {}
for K in K_LIST:
    recalls = [recall_at_k(g, K, "rating_prev", "y_true_rel") for _, g in test_pred.groupby("id_usuario")]
    ndcgs   = [ndcg_at_k(g,   K, "rating_prev", "y_true_rel") for _, g in test_pred.groupby("id_usuario")]
    cov     = coverage_at_k(test_pred, K, "rating_prev", "nombre_sitio")
    metrics_reg[K] = {"recall": float(np.nanmean(recalls)),
                      "ndcg":   float(np.nanmean(ndcgs)),
                      "coverage": cov}

import json
print("Top-K REGRESIÓN (global):", json.dumps(metrics_reg, indent=2, ensure_ascii=False))
with open("metrics_regresion_topk.json","w",encoding="utf-8") as f:
    json.dump(metrics_reg, f, ensure_ascii=False, indent=2)

# ================= FIN PARCHE =================



,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.3808,0.2256,0.4750,0.5995,0.1161,0.1252,9.6800
lightgbm,Light Gradient Boosting Machine,0.3808,0.2257,0.4751,0.5992,0.1161,0.1251,2.6340
catboost,CatBoost Regressor,0.3824,0.2279,0.4774,0.5953,0.1166,0.1255,17.7540
br,Bayesian Ridge,0.3857,0.2326,0.4822,0.5871,0.1182,0.1272,2.5380
lr,Linear Regression,0.3859,0.2328,0.4824,0.5867,0.1182,0.1273,2.4580
ridge,Ridge Regression,0.3859,0.2327,0.4824,0.5868,0.1182,0.1273,2.1920
rf,Random Forest Regressor,0.3872,0.2339,0.4836,0.5847,0.1179,0.1269,50.0320
omp,Orthogonal Matching Pursuit,0.3908,0.2389,0.4888,0.5758,0.1199,0.1291,2.1820
ada,AdaBoost Regressor,0.3972,0.2436,0.4936,0.5675,0.1202,0.1300,6.9540
et,Extra Trees Regressor,0.3994,0.2496,0.4996,0.5569,0.1215,0.1307,50.3700


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.3783,0.2232,0.4724,0.6028,0.1151,0.1239
1,0.3808,0.2266,0.4761,0.6023,0.1163,0.1250
2,0.3837,0.2277,0.4771,0.5972,0.1165,0.1262
3,0.3790,0.2217,0.4708,0.6025,0.1148,0.1240
4,0.3855,0.2315,0.4811,0.5878,0.1181,0.1278
Mean,0.3815,0.2261,0.4755,0.5985,0.1162,0.1254
Std,0.0028,0.0034,0.0036,0.0057,0.0012,0.0015


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.3782,0.2233,0.4726,0.6026,0.1151,0.1238
1,0.3818,0.2276,0.4770,0.6007,0.1165,0.1253
2,0.3836,0.2283,0.4779,0.5960,0.1168,0.1263
3,0.3788,0.2219,0.4711,0.6020,0.1148,0.1239
4,0.3851,0.2316,0.4813,0.5875,0.1181,0.1277
Mean,0.3815,0.2266,0.4760,0.5978,0.1163,0.1254
Std,0.0027,0.0035,0.0037,0.0056,0.0012,0.0015


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006520 seconds.
You can set `force_row_wise=true` to remove the overhead.

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.3771,0.2220,0.4712,0.6049,0.1149,0.1235
1,0.3807,0.2259,0.4753,0.6037,0.1161,0.1250
2,0.3832,0.2273,0.4768,0.5979,0.1166,0.1262
3,0.3776,0.2210,0.4702,0.6036,0.1147,0.1237
4,0.3838,0.2305,0.4802,0.5894,0.1181,0.1276
Mean,0.3805,0.2254,0.4747,0.5999,0.1161,0.1252
Std,0.0028,0.0035,0.0037,0.0058,0.0012,0.0015


Fitting 5 folds for each of 10 candidates, totalling 50 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.3770,0.2219,0.4711,0.6051,0.1148,0.1235
1,0.3800,0.2252,0.4745,0.6049,0.1159,0.1247
2,0.3827,0.2270,0.4764,0.5985,0.1165,0.1260
3,0.3776,0.2207,0.4698,0.6042,0.1146,0.1236
4,0.3832,0.2294,0.4790,0.5914,0.1176,0.1271
Mean,0.3801,0.2248,0.4742,0.6008,0.1159,0.1250
Std,0.0025,0.0032,0.0034,0.0053,0.0011,0.0014


Transformation Pipeline and Model Successfully Saved


Top-K REGRESIÓN (global): {
  "3": {
    "recall": 0.888768695456969,
    "ndcg": 0.8813271402023607,
    "coverage": 1.0
  },
  "5": {
    "recall": 0.9686028565816427,
    "ndcg": 0.9078232291455232,
    "coverage": 1.0
  },
  "10": {
    "recall": 0.9984819148637651,
    "ndcg": 0.9199722878177368,
    "coverage": 1.0
  }
}


In [11]:
import numpy as np
import pandas as pd
from pycaret.classification import load_model as load_cls, predict_model as predict_cls
from pycaret.regression import     load_model as load_reg,  predict_model as predict_reg

CLS = load_cls("modelo_cls_like_v2")
REG = load_reg("modelo_reg_rating_v2")
CAT = pd.read_csv(CAT_PATH, sep=SEP, encoding=ENC)

IGNORED_AT_TRAIN = ["id_usuario"]  # se ignoró en setup -> NO pasarla al pipeline

def _build_candidates(user: dict) -> pd.DataFrame:
    cand = CAT.copy()
    for k, v in user.items():
        cand[k] = v
    cand = normalize_columns(cand)
    cand = make_features(cand)                 # mismas features que en train
    return cand

def _prepare_for_pipeline(X: pd.DataFrame) -> pd.DataFrame:
    X = X.copy()
    # quitar columnas ignoradas en setup
    X.drop(columns=[c for c in IGNORED_AT_TRAIN if c in X.columns], inplace=True, errors="ignore")
    # (opcional) asegurar tipos como en train
    for c in CAT_COLS_X:
        if c in X.columns: X[c] = X[c].astype("string")
    for c in NUM_COLS_X:
        if c in X.columns: X[c] = pd.to_numeric(X[c], errors="coerce").astype("float64")
    return X

def _prob_like_from_hard_voting(pipeline, Xdf: pd.DataFrame) -> pd.Series:
    # 1) preprocesa con el pipeline sin el último paso (modelo)
    pre = pipeline[:-1]
    Xenc = pre.transform(Xdf)

    clf = pipeline.named_steps.get('trained_model', pipeline.steps[-1][1])
    proba_list = []

    for est in getattr(clf, 'estimators_', []):
        if hasattr(est, "predict_proba"):
            p = est.predict_proba(Xenc)
            classes = getattr(est, "classes_", None)
            if classes is not None:
                cls_list = list(classes)
                if 1 in cls_list: pos = cls_list.index(1)
                elif "1" in cls_list: pos = cls_list.index("1")
                else: pos = p.shape[1]-1
            else:
                pos = p.shape[1]-1
            proba_list.append(p[:, pos])
        elif hasattr(est, "decision_function"):
            df = est.decision_function(Xenc)
            if df.ndim == 1:
                proba_list.append(1.0/(1.0 + np.exp(-df)))            # logística
            else:
                col = 1 if df.shape[1] > 1 else 0
                proba_list.append(1.0/(1.0 + np.exp(-df[:, col])))

    if proba_list:
        return pd.Series(np.mean(np.column_stack(proba_list), axis=1), index=Xdf.index)

    # último recurso: usar predict_model por si expone alguna columna de score
    scored = predict_cls(CLS, data=Xdf, raw_score=True)
    if "Score_1" in scored.columns: return scored["Score_1"]
    if "Score" in scored.columns and "prediction_label" in scored.columns:
        return pd.Series(np.where(scored["prediction_label"]==1, scored["Score"], 1.0-scored["Score"]), index=Xdf.index)
    if "prediction_score" in scored.columns and "prediction_label" in scored.columns:
        return pd.Series(np.where(scored["prediction_label"]==1, scored["prediction_score"], 1.0-scored["prediction_score"]), index=Xdf.index)

    raise AttributeError("No hay forma de obtener probas del VotingClassifier (hard). Considera reentrenar con voting='soft'.")

def recomendar_top3_cls(user: dict) -> pd.DataFrame:
    X0 = _build_candidates(user)
    X  = _prepare_for_pipeline(X0)                 # << quita id_usuario y ajusta tipos

    # Intento directo (por si NO es 'hard'); si falla, uso agregación de base estimators
    try:
        proba = CLS.predict_proba(X)
        classes = getattr(CLS, "classes_", None)
        if classes is not None and (1 in list(classes) or "1" in list(classes)):
            pos = list(classes).index(1) if 1 in list(classes) else list(classes).index("1")
        else:
            pos = proba.shape[1]-1
        prob_like = pd.Series(proba[:, pos], index=X.index)
    except Exception:
        prob_like = _prob_like_from_hard_voting(CLS, X)

    scored = X0.assign(prob_like=prob_like.values)  # unimos a los campos legibles (tipo_sitio, etc.)

    cols = ["nombre_sitio","tipo_sitio","costo_entrada","accesibilidad_general",
            "afinidad_tipo","ratio_costo_presu","prob_like"]
    cols = [c for c in cols if c in scored.columns]
    return scored[cols].nlargest(3, "prob_like").reset_index(drop=True)

def recomendar_top3_reg(user: dict) -> pd.DataFrame:
    X0 = _build_candidates(user)
    X  = _prepare_for_pipeline(X0)                 # opcional (predict_model ya lo ignora)
    scored = predict_reg(REG, data=X)
    pred_col = "prediction_label" if "prediction_label" in scored.columns else ("Label" if "Label" in scored.columns else None)
    if pred_col is None:
        raise KeyError(f"No encuentro columna de predicción en regresión. Tengo: {list(scored.columns)[:20]}")
    scored = scored.rename(columns={pred_col: "rating_prev"})

    cols = ["nombre_sitio","tipo_sitio","costo_entrada","accesibilidad_general",
            "afinidad_tipo","ratio_costo_presu","rating_prev"]
    cols = [c for c in cols if c in X0.columns] + ["rating_prev"]
    out = X0.join(scored[["rating_prev"]]).nlargest(3, "rating_prev")[cols].reset_index(drop=True)
    return out



usuario_demo = {
    "id_usuario":"U_demo","edad":50,"nacionalidad":"Colombia","origen":"Bogotá",
    "tipo_turista_preferido":"aventura","compañia_viaje":"pareja",
    "frecuencia_viaje":2,"restricciones_movilidad":"ninguna",
    "presupuesto_estimado":230000,"sitios_visitados":6,
    "calificacion_sitios_previos":4.3,"tiempo_estancia_promedio":90,
    "epoca_visita":"fin_de_semana_puente"
}

print("Top-3 (clasificación):")
print(recomendar_top3_cls(usuario_demo).to_string(index=False))

print("\nTop-3 (regresión):")
print(recomendar_top3_reg(usuario_demo).to_string(index=False))



Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Top-3 (clasificación):
                    nombre_sitio tipo_sitio  costo_entrada accesibilidad_general  afinidad_tipo  ratio_costo_presu  prob_like
           Cascadas La Periquera   aventura          15000                  baja           0.95           0.434783   0.716560
                  Cañón La Honda   aventura          10000                  baja           0.95           0.289855   0.712285
Sendero Ecológico de Sutamarchán senderismo              0                 media           0.85           0.000000   0.656278

Top-3 (regresión):


                    nombre_sitio tipo_sitio  costo_entrada accesibilidad_general  afinidad_tipo  ratio_costo_presu  rating_prev
           Cascadas La Periquera   aventura          15000                  baja           0.95           0.434783     4.673013
                  Cañón La Honda   aventura          10000                  baja           0.95           0.289855     4.661522
Sendero Ecológico de Sutamarchán senderismo              0                 media           0.85           0.000000     4.375343


In [12]:
# Clasificación
clf = final_cls.named_steps["trained_model"]
print(clf)               # tipo (VotingClassifier)
print([type(e).__name__ for e in clf.estimators_])  # modelos base

# Regresión
reg = final_reg.named_steps["trained_model"]
print(reg)               # tipo (VotingRegressor)
print([type(e).__name__ for e in reg.estimators_])  # modelos base


KeyError: 'trained_model'

In [13]:
from sklearn.ensemble import VotingClassifier, VotingRegressor, StackingClassifier, StackingRegressor

def inspect_pycaret_pipeline(pipe):
    # 1) nombres de pasos
    step_names = [name for name, _ in pipe.steps]
    print("Pasos del Pipeline:", step_names)

    # 2) último paso = estimador final
    last_name, last_step = pipe.steps[-1]
    print(f"Paso final: {last_name}  →  {type(last_step).__name__}")

    # 3) según el tipo, muestra su composición
    if isinstance(last_step, (VotingClassifier, VotingRegressor)):
        # estimadores base (usamos estimators_ si está disponible; si no, estimators)
        base = getattr(last_step, "estimators_", None)
        if base is None:
            base = [est for _, est in getattr(last_step, "estimators", [])]
        print("Tipo Voting:", getattr(last_step, "voting", "—"))
        print("Estimadores base:", [type(e).__name__ for e in base])

    elif isinstance(last_step, (StackingClassifier, StackingRegressor)):
        base = getattr(last_step, "estimators_", None)
        if base is None:
            base = [est for _, est in getattr(last_step, "estimators", [])]
        meta = getattr(last_step, "final_estimator_", getattr(last_step, "final_estimator", None))
        print("Estimadores base (stacking):", [type(e).__name__ for e in base])
        print("Meta-modelo:", type(meta).__name__ if meta is not None else "—")

    else:
        print("Modelo único (no es ensamble).")

# Inspecciona tus modelos guardados:
inspect_pycaret_pipeline(CLS)   # clasificación
inspect_pycaret_pipeline(REG)   # regresión


Pasos del Pipeline: ['numerical_imputer', 'categorical_imputer', 'ordinal_encoding', 'onehot_encoding', 'rest_encoding', 'remove_multicollinearity', 'clean_column_names', 'actual_estimator']
Paso final: actual_estimator  →  VotingClassifier
Tipo Voting: hard
Estimadores base: ['GradientBoostingClassifier', 'RidgeClassifier', 'AdaBoostClassifier']
Pasos del Pipeline: ['numerical_imputer', 'categorical_imputer', 'ordinal_encoding', 'onehot_encoding', 'rest_encoding', 'remove_multicollinearity', 'clean_column_names', 'actual_estimator']
Paso final: actual_estimator  →  VotingRegressor
Tipo Voting: —
Estimadores base: ['GradientBoostingRegressor', 'LGBMRegressor', 'CatBoostRegressor']


In [14]:
_, est_final = CLS.steps[-1]
print(list(est_final.get_params().keys())[:20])  # algunas claves


['estimators', 'flatten_transform', 'n_jobs', 'verbose', 'voting', 'weights', 'Gradient Boosting Classifier', 'Ridge Classifier', 'Ada Boost Classifier', 'Gradient Boosting Classifier__ccp_alpha', 'Gradient Boosting Classifier__criterion', 'Gradient Boosting Classifier__init', 'Gradient Boosting Classifier__learning_rate', 'Gradient Boosting Classifier__loss', 'Gradient Boosting Classifier__max_depth', 'Gradient Boosting Classifier__max_features', 'Gradient Boosting Classifier__max_leaf_nodes', 'Gradient Boosting Classifier__min_impurity_decrease', 'Gradient Boosting Classifier__min_samples_leaf', 'Gradient Boosting Classifier__min_samples_split']


In [15]:
from pycaret.classification import (
    setup, compare_models, tune_model, blend_models,
    calibrate_model, finalize_model, save_model, pull
)

# Usa el mismo train_df y el MISMO setup que ya te funcionó (groupkfold + ignore id_usuario)
setup(
    data=train_df[["id_usuario"] + CAT_COLS_X + NUM_COLS_X + ["y_like"]],
    target="y_like",
    session_id=RANDOM_SEED,
    fold=5, fold_strategy="groupkfold", fold_groups="id_usuario",
    categorical_features=CAT_COLS_X,
    ignore_features=["id_usuario"],
    remove_multicollinearity=True, multicollinearity_threshold=0.95,
    imputation_type="simple",
    fix_imbalance=False,   # activa luego si lo necesitas
    verbose=False
)

# candidatos y tuning
cands = compare_models(n_select=5, sort="AUC")
tuned = [tune_model(m, optimize="AUC") for m in cands]
# quédate con modelos que soportan predict_proba (requisito para 'soft')
tuned = [m for m in tuned if hasattr(m, "predict_proba")]

# ensamble 'soft' (promedia probabilidades)
soft = blend_models(estimator_list=tuned, method="soft", choose_better=True)

# (opcional pero recomendado) calibra probabilidades
soft_cal = calibrate_model(soft, method="isotonic")

# cierra y guarda
final_soft = finalize_model(soft_cal)
save_model(final_soft, "modelo_cls_like_soft_v1")


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9077,0.9343,0.7363,0.7985,0.7661,0.7087,0.7096,3.1580
lightgbm,Light Gradient Boosting Machine,0.9071,0.9332,0.7367,0.7957,0.7650,0.7072,0.7081,1.0860
ridge,Ridge Classifier,0.9060,0.9331,0.7064,0.8115,0.7552,0.6974,0.7000,0.9300
lda,Linear Discriminant Analysis,0.9063,0.9330,0.7285,0.7978,0.7615,0.7033,0.7045,1.2700
ada,Ada Boost Classifier,0.9052,0.9325,0.7520,0.7789,0.7651,0.7057,0.7060,1.7380
catboost,CatBoost Classifier,0.9048,0.9324,0.7306,0.7901,0.7591,0.6999,0.7008,7.8960
rf,Random Forest Classifier,0.9042,0.9247,0.7242,0.7918,0.7564,0.6970,0.6981,2.2120
et,Extra Trees Classifier,0.9004,0.9207,0.7185,0.7795,0.7477,0.6858,0.6867,2.7100
lr,Logistic Regression,0.9008,0.9001,0.6744,0.8106,0.7361,0.6757,0.6801,5.6960
dt,Decision Tree Classifier,0.8573,0.7880,0.6696,0.6476,0.6583,0.5681,0.5684,1.0220


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9142,0.9384,0.7521,0.8094,0.7797,0.7265,0.7273
1,0.9032,0.9270,0.7249,0.7740,0.7486,0.6887,0.6893
2,0.9070,0.9348,0.7495,0.7956,0.7718,0.7135,0.7140
3,0.9083,0.9334,0.7348,0.8040,0.7679,0.7109,0.7120
4,0.9047,0.9352,0.7524,0.7846,0.7681,0.7082,0.7085
Mean,0.9075,0.9338,0.7427,0.7935,0.7672,0.7096,0.7102
Std,0.0038,0.0038,0.0110,0.0129,0.0102,0.0122,0.0122


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9124,0.9392,0.7377,0.8113,0.7728,0.7187,0.7199
1,0.9043,0.9276,0.7176,0.7831,0.7489,0.6899,0.6909
2,0.9064,0.9338,0.7314,0.8050,0.7664,0.7081,0.7094
3,0.9084,0.9337,0.7311,0.8070,0.7672,0.7103,0.7117
4,0.9045,0.9341,0.7487,0.7859,0.7668,0.7068,0.7072
Mean,0.9072,0.9337,0.7333,0.7985,0.7644,0.7068,0.7078
Std,0.0030,0.0037,0.0101,0.0116,0.0081,0.0094,0.0095


Fitting 5 folds for each of 10 candidates, totalling 50 fits


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9137,0.9394,0.7360,0.8180,0.7748,0.7216,0.7232
1,0.9029,0.9280,0.7069,0.7839,0.7434,0.6838,0.6852
2,0.9074,0.9340,0.7399,0.8036,0.7704,0.7126,0.7135
3,0.9082,0.9343,0.7284,0.8079,0.7661,0.7092,0.7106
4,0.9037,0.9349,0.7417,0.7870,0.7637,0.7033,0.7038
Mean,0.9072,0.9341,0.7306,0.8001,0.7637,0.7061,0.7073
Std,0.0038,0.0037,0.0127,0.0128,0.0108,0.0126,0.0127


Original model was better than the blended model, hence it will be returned. NOTE: The display metrics are for the blended model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9130,0.9393,0.7322,0.8177,0.7726,0.7190,0.7206
1,0.9042,0.9282,0.6979,0.7953,0.7434,0.6848,0.6870
2,0.9069,0.9340,0.7250,0.8113,0.7657,0.7078,0.7096
3,0.9073,0.9343,0.7403,0.7963,0.7672,0.7095,0.7102
4,0.9063,0.9354,0.7455,0.7950,0.7694,0.7107,0.7113
Mean,0.9075,0.9342,0.7282,0.8031,0.7637,0.7064,0.7077
Std,0.0029,0.0036,0.0167,0.0095,0.0104,0.0114,0.0111


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['edad', 'frecuencia_viaje',
                                              'presupuesto_estimado',
                                              'sitios_visitados',
                                              'calificacion_sitios_previos',
                                              'tiempo_estancia_promedio',
                                              'costo_entrada',
                                              'afluencia_promedio',
                                              'duracion_esperada',
                                              'admite_mascotas',
                                              'ratio_costo_presu',
                                              'afinidad_tipo'],
                                     transf...
                                                                    

In [25]:
from pycaret.classification import load_model
CLS_SOFT = load_model("modelo_cls_like_soft_v1")

def recomendar_top3_cls_soft(user: dict) -> pd.DataFrame:
    X0 = _build_candidates(user)
    X  = _prepare_for_pipeline(X0)          # quita id_usuario y asegura tipos
    proba = CLS_SOFT.predict_proba(X)       # ahora sí disponible
    pos = list(CLS_SOFT.classes_).index(1) if 1 in CLS_SOFT.classes_ else proba.shape[1]-1
    X0["prob_like"] = proba[:, pos]
    cols = ["nombre_sitio","tipo_sitio","costo_entrada","accesibilidad_general",
            "afinidad_tipo","ratio_costo_presu","prob_like"]
    return X0[cols].nlargest(3, "prob_like").reset_index(drop=True)

# probar:
print(recomendar_top3_cls_soft(usuario_demo))


Transformation Pipeline and Model Successfully Loaded
                nombre_sitio tipo_sitio  costo_entrada accesibilidad_general  \
0  Antigua Estación del Tren  histórico           5168                 media   
1    Claustro de San Agustín  histórico           5000                 media   
2            Museo del Fósil      museo          10000                  alta   

   afinidad_tipo  ratio_costo_presu  prob_like  
0            0.9           0.149797   0.957099  
1            0.9           0.144928   0.957099  
2            0.9           0.289855   0.920571  
